# CNN
## Convolutional Neural Network　畳込みニューラルネットワーク

### (1) 入力層
28x28=784次元のベクトルが入力ベクトルです。<br>
数字の画像を784ピクセルに分割し、1次元=1ピクセルで情報を代入しています。

### (2) 畳込み層1 -> プーリング層1
2層準備。ゼロパディング(zero padding)という補完機能を利用しています。<br>
ファイルの畳込みに対して、周縁データが欠落するのですが、その補完のためにパディングを用います。<br>
（パディングの話で推測できるように、畳込み層では、入力の次元圧縮を行いません）<br>
プーリング(pooling)では、畳込みで返された数値を圧縮します。<br>
ここでは、最大プーリング(max pooling)という、最も基本的な手法を用いています。<br>
プーリングでも、パディングを行うことがあります。<br>
（重ね合わせがずれる領域が存在することがあるため）

* ゼロパディング(zero padding)とは、画像や帳票などで数値を表現する際、<br>
書式で指定した桁数に満たない部分をゼロで埋めることである。<br>
例えば「123」を5桁で表す場合、2桁足りない部分をゼロで埋めて「00123」と表記する<br>
* 最大プーリング(max pooling)とは、設定範囲のピクセルの中で最大値を選択する

### (3) 畳込み層2 -> プーリング層2
(2)の繰り返しでさらに圧縮 -> この段階で7x7=49次元まで落とす予定です。

### (4) 全結合層(高密度結合層)
抽出したプーリング層からの出力層を入力層に送り、NNのような処理をかけます。<br>
活性化関数はReLUです。

### (5) クラス分類処理
ソフトマックス関数の計算 -> 交差エントロピー誤差関数による評価

https://qiita.com/icoxfog417/items/5fd55fad152231d706c2


In [13]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Build a Multilayer Convolutional Network

In [19]:
# Weight Initialization
# 適当にノイズを含んだ重み行列作成関数（ノイズは対称性の除去と勾配ゼロ防止のため）
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


# バイアス行列作成関数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


# Convolution and Pooling
# 2次元畳込み関数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


# 2x2マックスプーリング関数
def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


# Placeholders
# データ用可変2階テンソルを用意
x = tf.placeholder(tf.float32, shape=[None, 784])
# 正解用可変2階テンソルを用意
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# 画像をリシェイブ：第2引数は画像数(-1は元サイズを保存するように自動計算)、縦X横、チャンネル
x_image = tf.reshape(x, [-1, 28, 28, 1])

# First Convolutional Layer
# 1層目 畳み込み層
# 畳み込み層のフィルタ重み：引数はパッチサイズ縦、パッチサイズ横、入力チャンネル数、出力チャンネル数
W_conv1 = weight_variable([5, 5, 1, 32])
# 畳み込み層のバイアス
b_conv1 = bias_variable([32])
# 活性化関数ReLUでの畳み込み層を構築
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 2x2のマックスプーリング層を構築
h_pool1 = max_pool_2x2(h_conv1)

# Second Convolutional Layer
# 2層目 畳み込み層
# パッチサイズ縦、パッチサイズ横、入力チャンネル数、出力チャンネル数
# 5x5フィルタで64チャネルを出力
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer
# 全結合層
"""
オリジナル画像は28x28で、今回畳み込みでpadding='SAME'を指定しているため
プーリングでのみ画像サイズが変わる。
2x2プーリングで2x2でスライドも2x2なので
縦横ともに各層で半減する。
そのため、28/2/2=7が現在の画像サイズ

全結合層にするために、1階テンソルに変形。
画像サイズ縦と画像サイズ横とチャンネル数の積の次元
出力は1024(この辺は決め)、あとはsoftmax regressionと同じ
"""
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
# ドロップアウトを指定
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout Layer
# 出力層
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# 予測値
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

###  Train and Evaluate the Model

In [27]:
# 評価系の関数を用意
# Loss(cross_entropy) and optimizer(train_step)
cross_entropy = tf.reduce_mean()
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# テストのモデル
pred = tf.argmax(y_conv, 1)
true = tf.argmax(y_, 1)
correct_prediction = tf.equal(pred, true)

# 予測の精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

start = time.time()
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if i % 100 == 0:
        train_acc = accuracy.eval(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 1.0
        })
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels,
            keep_prob: 1.0
        })
        print "step % 5d" % (i), "train_acc={:5f}".format(
            train_acc), "test_acc={:5f}".format(test_acc)
end = time.time()
print "Process Time(s):{:.2f}".format(end - start)
print("test accuracy %g" % accuracy.eval(feed_dict={
    x: mnist.test.images,
    y_: mnist.test.labels,
    keep_prob: 1.0
}))

step     0 train_acc=0.220000 test_acc=0.170500
step   100 train_acc=0.900000 test_acc=0.868800
step   200 train_acc=0.920000 test_acc=0.913800
step   300 train_acc=0.920000 test_acc=0.933000
step   400 train_acc=0.960000 test_acc=0.939500
step   500 train_acc=0.940000 test_acc=0.949400
step   600 train_acc=1.000000 test_acc=0.958400
step   700 train_acc=0.980000 test_acc=0.956900
step   800 train_acc=0.940000 test_acc=0.956900
step   900 train_acc=0.920000 test_acc=0.962300
Process Time(s):97.10
test accuracy 0.9682
